# AMPLab 2025 Processing large datasets and Collaborative Filtering

Install dependencies in a separate cell, so that if people run it in jupyter with dependencies already present, they can choose to omit it.

We also install the `zstd` commandline tool to process our data files

In [1]:
%pip install implicit numpy scipy h5py

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install zstandard

  Using cached zstandard-0.23.0-cp310-cp310-win_amd64.whl.metadata (3.0 kB)
Using cached zstandard-0.23.0-cp310-cp310-win_amd64.whl (495 kB)


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import collections
import csv
import json
import os
import shutil
import glob
import zstandard as zstd

import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict

### Memory management
Here's a quick way of seeing what variables are using up a lot of memory.

```py
import sys
global_keys = list(globals().keys())
for k in global_keys:
  if not k.startswith("_"):
    size = sys.getsizeof(globals()[k])
    if size > 10000:
      print(f"{k}: {round(size / 1024 / 1024, 2)}MB")
```

You can delete any variable by running

```py
thevariable = None
```

which will cause python to garbage-collect it and free up the memory. It's worth doing this from time to time in colab if you are at risk of running out of memory in the runtime.


# Part 1: Data processing

## Step 1: Reading initial data files

In [2]:
# Where the original data files are - use a google drive shortcut so that you don't need to copy them
data_root = "F:/SMC/AMP Lab/Assignment_02/ListenBrainz/2025-materials"
# Local data directory for items that you don't want to lose
working_root = "F:/SMC/AMP Lab/Assignment_02/amplab-working"
# Temporary location that you can use while the colab is running
scratch_root = "F:/SMC/AMP Lab/Assignment_02/amplab-scratch"
os.makedirs(working_root, exist_ok=True)
os.makedirs(scratch_root, exist_ok=True)

Double-check the data directory to see what's there

In [3]:
from pathlib import Path
for file in Path(data_root).glob('*'):
    print(f"{file.name:50} {file.stat().st_size/1024/1024:.2f} MB")

1.listens.zst                                      755.02 MB
10.listens.zst                                     617.84 MB
11.listens.zst                                     618.28 MB
12.listens.zst                                     635.66 MB
2.listens.zst                                      710.10 MB
3.listens.zst                                      737.87 MB
4.listens.zst                                      731.43 MB
5.listens.zst                                      707.73 MB
6.listens.zst                                      626.55 MB
7.listens.zst                                      644.80 MB
8.listens.zst                                      596.39 MB
9.listens.zst                                      589.38 MB
canonical_musicbrainz_data.csv.zst                 1478.30 MB
canonical_recording_redirect.csv.zst               236.33 MB
listenbrainz_model.py                              0.00 MB
listenbrainz_msid_mapping.csv-001.zst              3465.63 MB
musicbrainz_artist.csv  

The data files are compressed using the zstandard format. Colab has a "local disk" that we can use temporarily. This is not saved when the notebook notebook is closed, but is faster than reading directly from google drive. We can copy a file to our drive and uncompress it to inspect it.

We know that a data file is in "jsonlines" format, where each line of the file is a json document. We can inspect the first line to see the structure of this document, and see that we want to read the `user_id` and `recording_msid` fields.

In [ ]:
with open(os.path.join(scratch_root, "1.listens")) as fp:
  line = fp.readline()
  line_json = json.loads(line)
  print(json.dumps(line_json, indent=4))

We can read a file one line at a time, parse the json, and save the user id and recording msid.
Note that if you do this for all files then the `data` variable may cause you to run out of memory. Consider this in your work

In [5]:
def process_listen_files():
    all_data = []
    # Process all listen files (1-12 for each month)
    for file_num in range(1, 13):
        input_file = f"{file_num}.listens.zst"
        compressed_path = Path(data_root) / input_file
        decompressed_path = Path(scratch_root) / f"{file_num}.listens"
        
        # Decompress the file
        with open(compressed_path, 'rb') as f_in:
            with open(decompressed_path, 'wb') as f_out:
                dctx = zstd.ZstdDecompressor()
                dctx.copy_stream(f_in, f_out)
        
        # Process the decompressed file
        with open(decompressed_path, 'r', encoding='utf-8') as fp:
            for line in fp:
                try:
                    j = json.loads(line)
                    user_id = j['user_id']
                    recording_msid = j['recording_msid']
                    all_data.append([user_id, recording_msid])
                except (json.JSONDecodeError, KeyError):
                    continue
                    
        # Clean up decompressed file
        decompressed_path.unlink()
        
        # Show progress
        print(f"Processed file {file_num}/12 - Current total records: {len(all_data):,}")
    
    return all_data

In [39]:
# Process all files
data = process_listen_files()
print(f"\nTotal records processed: {len(data):,}")

Processed file 1/12 - Current total records: 8,839,350
Processed file 2/12 - Current total records: 17,075,471
Processed file 3/12 - Current total records: 25,571,926
Processed file 4/12 - Current total records: 33,956,587
Processed file 5/12 - Current total records: 42,132,406
Processed file 6/12 - Current total records: 49,531,522
Processed file 7/12 - Current total records: 57,072,750
Processed file 8/12 - Current total records: 64,251,918
Processed file 9/12 - Current total records: 71,446,889
Processed file 10/12 - Current total records: 78,870,406
Processed file 11/12 - Current total records: 86,213,100
Processed file 12/12 - Current total records: 93,859,039

Total records processed: 93,859,039


In [40]:
# Save the data that we extracted to a single file, in case the colab runtime quits
# The arguments to `open` are best practices when reading CSV files in python, although I don't always use them
# https://docs.python.org/3/library/csv.html#csv.reader
with open(os.path.join(working_root, "userid-msid.csv"), "w", newline='', encoding='utf-8') as fp:
    w = csv.writer(fp)
    w.writerows(data)

In [6]:
# Load the data from the previous cell again, if we stopped at this point
data = []
with open(os.path.join(working_root, "userid-msid.csv"), "r", newline='', encoding='utf-8') as fp:
    r = csv.reader(fp)
    for line in r:
        data.append(line)

In [14]:
# Double-check the total number of lines of data we have
line_count = sum(1 for _ in open(os.path.join(working_root, "userid-msid.csv"), 'r', encoding='utf-8'))
print(f"Total number of lines in file: {line_count}")

Total number of lines in file: 93859039


## Step 2: Mapping MSID to MBID

In [7]:
listenbrainz_msid_mapping_fn = shutil.copy2(os.path.join(data_root, "listenbrainz_msid_mapping.csv-001.zst"), scratch_root)
compressed_file = Path(scratch_root) / Path(data_root).glob("*mapping*.zst").__next__().name
with open(compressed_file, 'rb') as src, open(compressed_file.with_suffix(''), 'wb') as dst:
    zstd.ZstdDecompressor().copy_stream(src, dst)

Try and load the mapping CSV into memory. Ideally I want to have a dictionary such as:

```py
mapping = {
  "msid1": "mbid1",
  "msid2": "mbid"
}
```

which would allow us to look up an msid in the `data` list above, with just

```py
mbid = mapping[data[0][1]]
```

Unfortunately, this CSV file is too large for colab, and it will run out of memory before loading the entire file.

However, we can see that the size of the mapping file (111 million rows) is much larger than the number of unique MSIDs that we have in the data file

In [8]:
# On colab this cell will run out of memory
mapping = {}
with open(os.path.join(scratch_root, "listenbrainz_msid_mapping.csv-001")) as fp:
    r = csv.reader(fp)
    # omit header
    next(r)
    for line in r:
        mapping[line[0]] = line[1]

In [9]:
mapping_file = Path(scratch_root) / "listenbrainz_msid_mapping.csv-001"
with open(mapping_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 40:
            break
        print(line.strip())

recording_msid,recording_mbid,match_type
13ca445f-c0dd-4f64-8726-7da78a3821aa,,no_match
54a40ef8-6bfe-4803-b74a-7b93885c2f01,,no_match
21c07966-97c6-4e02-a575-e1b2fadf0d34,,no_match
3b438b25-b9ad-480f-b248-bd06281919e0,,no_match
26d8cc2b-c249-4b70-9f16-4eea1419303c,,no_match
bf3fe1e0-7ae6-4e57-a38e-6ac7d42d33c3,,no_match
937d9d97-ba8b-4ff4-9af7-1b4a485945c1,,no_match
0e9230f2-1d9d-47ab-a44f-dbc788cfbdf1,,no_match
2c9111a7-96e6-4083-8b45-78cadb11796e,,no_match
012868b2-f6d7-40e6-80a4-d89d077e3d9e,,no_match
03feccff-3632-4acf-974b-b787ff7e9bbf,,no_match
364902a8-1067-4804-b8be-d6b801ee4179,,no_match
13a44caa-8109-46ef-ac25-0e89785bdd18,,no_match
273849c3-b1d7-4656-bf67-748c6cac2179,,no_match
da3aea98-c857-4eb7-b4d6-cce8aa0318d7,,no_match
c514427c-89e1-4746-86bb-49f5d5c8e04a,,no_match
67c2a304-1ff5-44ea-a076-9d797ba15c28,,no_match
127b9ad4-20d6-49e0-ab82-5aad8d27b5e1,,no_match
703adaef-f000-4e6c-bd96-6c9c299af991,,no_match
6848ccaf-4942-4fcf-bde3-fbe4e86ede21,,no_match
562560cc-a8a0-4857-

In [10]:
mapping_file = Path(scratch_root) / "listenbrainz_msid_mapping.csv-001"
with open(mapping_file, 'r', encoding='utf-8') as f:
    line_count = sum(1 for _ in f)
print(f"Number of lines in {mapping_file.name}: {line_count:,}")

Number of lines in listenbrainz_msid_mapping.csv-001: 111,339,170


In [11]:
unique_msids = set(line[1] for line in data)
print(f"Unique msids: {len(unique_msids)}")
# Note that this is even smaller than the 8.8m lines in the file `userid-msid.csv`, because we're removing duplicates.

Unique msids: 16684273


Therefore, we can read the mapping file one line at a time and make a new small mapping file containing only the MSID values that we need.

**Trick:** This is one of the tricks to work with large amounts of data without loading it into memory. You can read the file one line at a time (python does this if you iterate over a csv reader such as `for line in r`), or you could use `fp.readline`. You could also do a variant of this by reading a data frame in pandas in chunks (e.g. 100,000 lines at a time), and processing each chunk before moving on to the next one.

I decided to read all lines in the mapping, but you could also decide to use only exact match or high quality results.
If you look at the mapping file you can see that the 3rd column contains a "match_type" field representing the quality of the match. This can be "exact_match", "high_quality", "med_quality", "low_quality", or "no_match"

In [12]:
mapping_file = Path(scratch_root) / "listenbrainz_msid_mapping.csv-001"

with open(mapping_file, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    match_types = {row[2] for _, row in zip(range(1000), reader) if len(row) > 2}

print("Unique match types:")
print('\n'.join(f"- {t}" for t in sorted(match_types)))

Unique match types:
- exact_match
- high_quality
- low_quality
- med_quality
- no_match


In [15]:
# Here we don't even need to store the entire contents of these files.
# Instead we just read the first file 1 line and a time and write a new line directly to the 2nd file if needed.
with open(os.path.join(scratch_root, "listenbrainz_msid_mapping.csv-001")) as r_fp, open(os.path.join(scratch_root, "small_msid_mapping.csv-001"), "w") as w_fp:
    r = csv.reader(r_fp)
    w = csv.writer(w_fp)
    header = next(r)
    w.writerow(header)
    for line in r:
      if line[0] in unique_msids and line[2] != "no_match":
        w.writerow(line)

In [ ]:
# Define input and output files
input_file = Path(scratch_root) / "listenbrainz_msid_mapping.csv-001"
output_file = Path(scratch_root) / "high_quality_mapping.csv"

# Process the mapping file, filtering for high-quality matches
matched = 0
with open(input_file, 'r', encoding='utf-8') as r_fp, \
     open(output_file, 'w', newline='', encoding='utf-8') as w_fp:
    reader = csv.reader(r_fp)
    writer = csv.writer(w_fp)
    
    # Copy header
    writer.writerow(next(reader))
    
    # Filter and write only high-quality and exact matches for our MSIDs
    for line in reader:
        if (line[0] in unique_msids and 
            line[2] in ("exact_match", "high_quality")):
            writer.writerow(line)
            matched += 1
            if matched % 100000 == 0:  # Simple progress indicator
                print(f"Processed {matched:,} matches")

print(f"\nStats:")
print(f"Total unique MSIDs: {len(unique_msids):,}")
print(f"High-quality matches found: {matched:,}")
print(f"Coverage: {matched/len(unique_msids)*100:.2f}%")

Processed 100,000 matches
Processed 200,000 matches
Processed 300,000 matches
Processed 400,000 matches
Processed 500,000 matches
Processed 600,000 matches
Processed 700,000 matches
Processed 800,000 matches
Processed 900,000 matches
Processed 1,000,000 matches
Processed 1,100,000 matches
Processed 1,200,000 matches
Processed 1,300,000 matches
Processed 1,400,000 matches
Processed 1,500,000 matches
Processed 1,600,000 matches
Processed 1,700,000 matches
Processed 1,800,000 matches
Processed 1,900,000 matches
Processed 2,000,000 matches
Processed 2,100,000 matches
Processed 2,200,000 matches
Processed 2,300,000 matches
Processed 2,400,000 matches
Processed 2,500,000 matches
Processed 2,600,000 matches
Processed 2,700,000 matches
Processed 2,800,000 matches
Processed 2,900,000 matches
Processed 3,000,000 matches
Processed 3,100,000 matches
Processed 3,200,000 matches
Processed 3,300,000 matches
Processed 3,400,000 matches
Processed 3,500,000 matches
Processed 3,600,000 matches
Processed 

Now that we have a small mapping file, we know that it will fit in memory. Until now, `data` has been in memory (a list of userid/msid pairs).

We can free up the `data` variable from memory, load the mapping, and then process the "userid-msid.csv" file, one line at a time, immediately writing out a new file with the relevant MBID.

In [23]:
# Process the filtered mapping file
smallmapping = {}
with open(Path(scratch_root) / "small_msid_mapping.csv-001", 'r', encoding='utf-8') as fp:
    reader = csv.reader(fp)
    next(reader)  # Skip header
    for line in reader:
        # Only process lines that have at least 2 columns
        if len(line) >= 2:
            smallmapping[line[0]] = line[1]

print(f"Loaded {len(smallmapping):,} mappings")

# Verify a few entries
print("\nFirst few mappings:")
for i, (msid, mbid) in enumerate(list(smallmapping.items())[:5]):
    print(f"MSID: {msid} -> MBID: {mbid}")

Loaded 11,351,391 mappings

First few mappings:
MSID: 61612055-12bd-4225-8f11-b8036021d72a -> MBID: 66a08ebb-1d9c-4434-bd23-806dda6d2a8d
MSID: 5ec25683-f439-42ab-86e9-32b3c9611cdd -> MBID: e3a5f4f5-2648-4d2a-861f-19524bd558c1
MSID: 01c79a31-a6de-4a56-9c95-410a3106af6d -> MBID: f32d973a-9aef-4d95-ae32-93d75618219b
MSID: 881427b4-cfa5-4223-8fa3-288aa944525c -> MBID: 43a21737-0462-423d-9105-c5f63d67b5b0
MSID: 6b245bfc-0064-473d-8305-bb51467c4d12 -> MBID: 39db4cae-d9d1-4807-8be6-49e72bccb3da


# Canonical Redirect Processing
Process canonical redirects for MBIDs

In [ ]:
# decompress the canonical redirect file
canonical_redirect_zst = Path(data_root) / "canonical_recording_redirect.csv.zst"
canonical_redirect_csv = Path(scratch_root) / "canonical_recording_redirect.csv"

if not canonical_redirect_csv.exists():
    with open(canonical_redirect_zst, 'rb') as src, open(canonical_redirect_csv, 'wb') as dst:
        zstd.ZstdDecompressor().copy_stream(src, dst)

In [ ]:
# Create final mapping file
final_mapping_file = Path(scratch_root) / "final_canonical_mapping.csv"

def process_canonical_redirects():
    redirects_processed = 0
    mappings_processed = 0
    
    # Open output file for writing canonical mappings
    with open(final_mapping_file, 'w', newline='', encoding='utf-8') as out_fp:
        writer = csv.writer(out_fp)
        writer.writerow(['msid', 'canonical_mbid'])  # Write header
        
        # Process redirects in chunks
        chunk_size = 100000
        current_chunk = {}
        
        # Read canonical redirects and process in chunks
        with open(canonical_redirect_csv, 'r', encoding='utf-8') as redirect_fp:
            redirect_reader = csv.reader(redirect_fp)
            next(redirect_reader)  # Skip header
            
            # Process smallmapping entries
            for msid, mbid in smallmapping.items():
                # Read next chunk of redirects
                if len(current_chunk) == 0:
                    current_chunk = {}
                    for _ in range(chunk_size):
                        try:
                            line = next(redirect_reader)
                            if len(line) >= 2:
                                current_chunk[line[0]] = line[1]
                                redirects_processed += 1
                        except StopIteration:
                            break
                
                # Look up in current chunk or use original
                canonical_mbid = current_chunk.get(mbid, mbid)
                writer.writerow([msid, canonical_mbid])
                mappings_processed += 1
                
                if mappings_processed % 100000 == 0:
                    print(f"Processed {mappings_processed:,} mappings")

    return redirects_processed, mappings_processed



In [27]:
# Process the mappings
redirects_count, final_count = process_canonical_redirects()

print(f"\nStats:")
print(f"Redirects processed: {redirects_count:,}")
print(f"Final mappings created: {final_count:,}")

# Verify the output file
print(f"\nFirst few lines of final mapping:")
with open(final_mapping_file, 'r', encoding='utf-8') as f:
    for _ in range(5):
        print(f.readline().strip())

Processed 100,000 mappings
Processed 200,000 mappings
Processed 300,000 mappings
Processed 400,000 mappings
Processed 500,000 mappings
Processed 600,000 mappings
Processed 700,000 mappings
Processed 800,000 mappings
Processed 900,000 mappings
Processed 1,000,000 mappings
Processed 1,100,000 mappings
Processed 1,200,000 mappings
Processed 1,300,000 mappings
Processed 1,400,000 mappings
Processed 1,500,000 mappings
Processed 1,600,000 mappings
Processed 1,700,000 mappings
Processed 1,800,000 mappings
Processed 1,900,000 mappings
Processed 2,000,000 mappings
Processed 2,100,000 mappings
Processed 2,200,000 mappings
Processed 2,300,000 mappings
Processed 2,400,000 mappings
Processed 2,500,000 mappings
Processed 2,600,000 mappings
Processed 2,700,000 mappings
Processed 2,800,000 mappings
Processed 2,900,000 mappings
Processed 3,000,000 mappings
Processed 3,100,000 mappings
Processed 3,200,000 mappings
Processed 3,300,000 mappings
Processed 3,400,000 mappings
Processed 3,500,000 mappings
Pro

In [28]:
# Decompress canonical musicbrainz data
canonical_data_zst = Path(data_root) / "canonical_musicbrainz_data.csv.zst"
canonical_data_csv = Path(scratch_root) / "canonical_musicbrainz_data.csv"

if not canonical_data_csv.exists():
    print("Decompressing canonical musicbrainz data...")
    with open(canonical_data_zst, 'rb') as src, open(canonical_data_csv, 'wb') as dst:
        zstd.ZstdDecompressor().copy_stream(src, dst)

Decompressing canonical musicbrainz data...


In [29]:
# Look at the first few lines
with open(canonical_data_csv, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    print("File structure:")
    print(f"Header: {header}")
    print("\nFirst row example:")
    print(next(reader))

File structure:
Header: ['id', 'artist_credit_id', 'artist_mbids', 'artist_credit_name', 'release_mbid', 'release_name', 'recording_mbid', 'recording_name', 'combined_lookup', 'score']

First row example:
['1', '1', '89ad4ac3-39f7-470e-963a-56509c546377', 'Various Artists', '4fd4f7ee-cee8-47fd-84d2-8d65e74bd8f7', 'Nadal en galego', '00b1a29d-ad9e-4b64-aed6-281f69f628ae', 'Catro Mancebos', 'variousartistscatromancebos', '91870']


# Artist information processing

- In this step I chose to take the first artist when multiple artists exist
- Create "recording_artist_mapping.csv" containing:
    - Recording MBID
    - Artist MBID
    - Artist name

In [ ]:
def process_recording_to_artist_mapping():
    """Process canonical data to map recordings to their first artists"""
    
    output_file = Path(scratch_root) / "recording_artist_mapping.csv"
    processed = 0
    
    with open(canonical_data_csv, 'r', encoding='utf-8') as f_in, \
         open(output_file, 'w', newline='', encoding='utf-8') as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        
        # Write header
        header = next(reader)
        writer.writerow(['recording_mbid', 'artist_mbid', 'artist_name'])
        
        for row in reader:
            if len(row) < 8:  # Ensure we have all needed fields
                continue
                
            recording_mbid = row[6]
            artist_mbids = row[2].split(';') if row[2] else []
            artist_name = row[3]
            
            # Take only the first artist
            if artist_mbids:
                writer.writerow([recording_mbid, artist_mbids[0], artist_name])
            
            processed += 1
            if processed % 100000 == 0:
                print(f"Processed {processed:,} recordings")
    
    return output_file

In [31]:
# Process the recording to artist mapping
recording_artist_file = process_recording_to_artist_mapping()

# Verify the output
print("\nVerifying mapping file:")
with open(recording_artist_file, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)  # Skip header
    
    print("First 5 mappings:")
    for _ in range(5):
        row = next(reader)
        print(f"Recording: {row[0]} -> Artist: {row[1]} ({row[2]})")

Processed 100,000 recordings
Processed 200,000 recordings
Processed 300,000 recordings
Processed 400,000 recordings
Processed 500,000 recordings
Processed 600,000 recordings
Processed 700,000 recordings
Processed 800,000 recordings
Processed 900,000 recordings
Processed 1,000,000 recordings
Processed 1,100,000 recordings
Processed 1,200,000 recordings
Processed 1,300,000 recordings
Processed 1,400,000 recordings
Processed 1,500,000 recordings
Processed 1,600,000 recordings
Processed 1,700,000 recordings
Processed 1,800,000 recordings
Processed 1,900,000 recordings
Processed 2,000,000 recordings
Processed 2,100,000 recordings
Processed 2,200,000 recordings
Processed 2,300,000 recordings
Processed 2,400,000 recordings
Processed 2,500,000 recordings
Processed 2,600,000 recordings
Processed 2,700,000 recordings
Processed 2,800,000 recordings
Processed 2,900,000 recordings
Processed 3,000,000 recordings
Processed 3,100,000 recordings
Processed 3,200,000 recordings
Processed 3,300,000 record

# Final data creation
- Create a final data file that contains:
    - user_id
    - artist_id
    - artist_name
    - play count

In [32]:
def create_artist_name_mapping():
    """Create a mapping of artist IDs to names"""
    artist_names = {}
    artist_file = Path(data_root) / "musicbrainz_artist.csv"
    
    with open(artist_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        
        for row in reader:
            if len(row) >= 2:
                artist_id = row[0]
                artist_name = row[1]
                artist_names[artist_id] = artist_name
    
    return artist_names

In [33]:
# Create artist name mapping
artist_names = create_artist_name_mapping()
print(f"\nLoaded {len(artist_names):,} artist names")


Loaded 2,531,408 artist names


In [ ]:
def create_user_artist_counts():
    """Create a dictionary of {(user_id, artist_id): play_count}"""
    user_artist_plays = defaultdict(int)
    processed = 0
    skipped = 0
    
    # First, load recording to artist mapping into memory
    recording_to_artist = {}
    with open(Path(scratch_root) / "recording_artist_mapping.csv", 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            if len(row) >= 2:  # Ensure we have recording_mbid and artist_mbid
                recording_to_artist[row[0]] = row[1]  # Map recording_mbid to artist_mbid
    
    print(f"Loaded {len(recording_to_artist):,} recording-artist mappings")
    
    # Process listen data
    with open(Path(working_root) / "userid-msid.csv", 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        
        for row in reader:
            if len(row) < 2:
                continue
                
            user_id, msid = row
            
            # Get canonical recording MBID
            mbid = smallmapping.get(msid)
            if not mbid:
                skipped += 1
                continue
            
            # Get artist ID
            artist_id = recording_to_artist.get(mbid)
            if artist_id:
                user_artist_plays[(user_id, artist_id)] += 1
            
            processed += 1
            if processed % 100000 == 0:
                print(f"Processed {processed:,} listens")
    
    return user_artist_plays, processed, skipped

In [45]:
def save_play_counts(user_artist_plays, artist_names):
    """
    Save user-artist play counts to CSV file
    Args:
        user_artist_plays: Dictionary of {(user_id, artist_id): play_count}
        artist_names: Dictionary of artist_id to artist_name mappings
    Returns:
        Path to output file
    """
    output_file = Path(working_root) / "user_artist_playcounts.csv"
    saved_count = 0
    
    try:
        with open(output_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['user_id', 'artist_id', 'artist_name', 'play_count'])
            
            for (user_id, artist_id), count in user_artist_plays.items():
                artist_name = artist_names.get(artist_id, "Unknown")
                writer.writerow([user_id, artist_id, artist_name, count])
                saved_count += 1
                
                if saved_count % 100000 == 0:
                    print(f"Saved {saved_count:,} play counts")
        
        print(f"\nSuccessfully saved {saved_count:,} play counts to {output_file}")
        return output_file
    
    except Exception as e:
        print(f"Error saving play counts: {str(e)}")
        return None

In [ ]:
# Create user-artist play counts
print("Creating user-artist play counts...")
user_artist_plays, processed, skipped = create_user_artist_counts()

print(f"\nProcessing complete:")
print(f"Processed: {processed:,} listens")
print(f"Skipped: {skipped:,} listens")
print(f"Unique user-artist pairs: {len(user_artist_plays):,}")


In [46]:
# Save the play counts
if output_file := save_play_counts(user_artist_plays, artist_names):
    # Verify the output
    print("\nVerifying saved data:")
    with open(output_file, 'r', encoding='utf-8') as f:
        for _ in range(6):  # Header + 5 rows
            print(f.readline().strip())

Saved 100,000 play counts
Saved 200,000 play counts
Saved 300,000 play counts
Saved 400,000 play counts
Saved 500,000 play counts
Saved 600,000 play counts
Saved 700,000 play counts
Saved 800,000 play counts
Saved 900,000 play counts
Saved 1,000,000 play counts
Saved 1,100,000 play counts
Saved 1,200,000 play counts
Saved 1,300,000 play counts
Saved 1,400,000 play counts
Saved 1,500,000 play counts
Saved 1,600,000 play counts
Saved 1,700,000 play counts
Saved 1,800,000 play counts
Saved 1,900,000 play counts
Saved 2,000,000 play counts
Saved 2,100,000 play counts
Saved 2,200,000 play counts
Saved 2,300,000 play counts
Saved 2,400,000 play counts
Saved 2,500,000 play counts
Saved 2,600,000 play counts
Saved 2,700,000 play counts
Saved 2,800,000 play counts
Saved 2,900,000 play counts
Saved 3,000,000 play counts
Saved 3,100,000 play counts
Saved 3,200,000 play counts
Saved 3,300,000 play counts
Saved 3,400,000 play counts
Saved 3,500,000 play counts
Saved 3,600,000 play counts
Saved 3,70

In [4]:
# Check contents of user-artist playcounts file
df = pd.read_csv(Path(working_root) / "user_artist_playcounts.csv")
print("Data Structure:")
print(df.head())
print(f"\nTotal Entries: {len(df):,}")

Data Structure:
   user_id                             artist_id          artist_name  \
0    24076  29266b3d-b5ae-4d09-b721-326246adf68f       In This Moment   
1    22845  744b52c8-509b-4451-abfd-a17d18d4bd1d  Vince Guaraldi Trio   
2     2966  b7539c32-53e7-4908-bda3-81449c367da6         Lana Del Rey   
3    31175  875203e1-8e58-4b86-8dcb-7190faf411c5              J. Cole   
4     4942  84825fb6-c98c-4b43-a184-c7f70619f355            Roosevelt   

   play_count  
0          75  
1          11  
2         143  
3          43  
4         144  

Total Entries: 7,947,518



**Preparing data according to listenbrainz model requirements**

Since the model only expects user ID, artist and plays. The dataset structure is modified accordingly.


In [ ]:
def prepare_data_for_model():
    
    input_file = Path(working_root) / "user_artist_playcounts.csv"
    output_file = Path(working_root) / "userid-artist-counts.csv"
    
    # Create simplified version with only required columns
    with open(input_file, 'r', encoding='utf-8') as f_in, \
         open(output_file, 'w', newline='', encoding='utf-8') as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)
        
        # Skip original header
        next(reader)
        
        # Write header
        writer.writerow(['user', 'artist', 'plays'])
        
        # Process rows - keep only user_id, artist_id, and play_count
        for row in reader:
            writer.writerow([row[0], row[1], row[3]])
    
    # Verify
    df = pd.read_csv(output_file)
    print("New data structure:")
    print(df.head())
    print(f"\nTotal rows: {len(df):,}")

    return output_file

In [15]:
# Create model-compatible data file
model_input_file = prepare_data_for_model()

New data structure:
    user                                artist  plays
0  24076  29266b3d-b5ae-4d09-b721-326246adf68f     75
1  22845  744b52c8-509b-4451-abfd-a17d18d4bd1d     11
2   2966  b7539c32-53e7-4908-bda3-81449c367da6    143
3  31175  875203e1-8e58-4b86-8dcb-7190faf411c5     43
4   4942  84825fb6-c98c-4b43-a184-c7f70619f355    144

Total rows: 7,947,518


---

# Part 2: Collaborative filtering model

This part of the assignment is based on the [implicit tutorial](https://benfred.github.io/implicit/tutorial_lastfm.html) and builds a collaborative filtering model. We reuse as much code as possible from this library.

We can find this source code on the [project's github page](https://github.com/benfred/implicit), specifically https://github.com/benfred/implicit/blob/main/implicit/datasets/lastfm.py has code which builds the model based on the last.fm dataset.

We've extracted the relevant code from this file and modified it for this task. It's in `listenbrainz_model.py`

## Step 6: Generate matrix

In [3]:
# Add data_root to the python path so that we can load the provided file.
data_root_path = "amplab-working"
# You might want to move this file somewhere else and edit it, update the path as necessary
import sys
sys.path.append(data_root_path)

In [4]:
import listenbrainz_model as lb

c:\Users\madha\anaconda3\envs\mirenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
matrix_artists, matrix_users, plays = lb.load_data_matrix(os.path.join(working_root, "userid-artist-counts.csv"))

## Step 7: Build model

In [6]:
model = lb.build_model(plays)

c:\Users\madha\anaconda3\envs\mirenv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 24 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 15/15 [00:25<00:00,  1.72s/it]


We used Artist MBIDs in our data file in order to be unique, so we need one final mapping which allows us to go from an artist MBID to a name.

# Artist similarity

In [7]:
artist_map = lb.get_artist_map(os.path.join(data_root, "musicbrainz_artist.csv"))
def format_artist(artist_id):
    return f"""<a href="https://musicbrainz.org/artist/{artist_id}">{artist_map.get(artist_id, "unknown")}</a> ({artist_id})"""

**Recommending similar artists**

Finding and recommending similar tracks based on a particular artist index

In [8]:
from IPython.display import HTML

def find_similar_artists(artist_mbid: str, N: int = 10) -> HTML:
    # Get artist name from mapping
    artist_name = artist_map.get(artist_mbid, "Unknown Artist")
    
    # Get similar artists
    artist_idx = lb.artist_index(matrix_artists, artist_mbid)
    ids, scores = model.similar_items(artist_idx, N=N)
    
    similar_artists = pd.DataFrame({
        "Artist": [format_artist(a) for a in matrix_artists[ids]],
        "Similarity Score": scores.round(3)
    })
    
    # HTML output
    content_output = f"""
    <h3>Similar artists to {artist_name}</h3>
    <p><small>MBID: {artist_mbid}</small></p>
    {similar_artists.to_html(escape=False, index=False)}
    """
    
    return HTML(content_output)

In [12]:
# Find similar artists using artist IDs
pinkfloyd_mbid = "83d91898-7763-47d7-b03b-b92132375c47"
jeremyzucker_mbid = 'e116e0be-c371-4d57-bc09-e2d762d82540'
coldplay_mbid = 'cc197bad-dc9c-440d-a5b5-d52ba2e14234'
the1975_mbid =  '5b6ebfe0-f72b-4902-bba9-74c8af0f1af0'
illenium_mbid = '5f43abf6-92a5-468a-a633-b73f94627972'
arrahman_mbid = 'e0bba708-bdd3-478d-84ea-c706413bedab'
anirudh_mbid = 'cf76861a-c1f5-456d-8d17-5994d05eecf8'

find_similar_artists(arrahman_mbid, N=15)

Artist,Similarity Score
A. R. Rahman (e0bba708-bdd3-478d-84ea-c706413bedab),1.000
Lata Mangeshkar (aeb71bd8-447d-4415-8ea1-2b7d664f67e1),0.850
Asha Bhosle (79c5547a-e098-495c-8dac-7e99546aa46b),0.835
Rahat Fateh Ali Khan (014431e3-c5a3-4a57-b86b-fe8f2e3253ff),0.816
Kishore Kumar (793b8f58-80ab-49b3-b5ae-d94034dab10c),0.807
Shreya Ghoshal (a8740949-50a8-4e71-8133-17d31b7cf69c),0.805
Arijit Singh (ed3f4831-e3e0-4dc0-9381-f5649e9df221),0.805
Atif Aslam (2c26fddb-3926-4004-ae27-22a3896a4f26),0.797
Sonu Nigam (93622908-0806-4173-94c1-9e42597af011),0.795
Mohit Chauhan (1dd28f27-4ab3-4a3f-8174-4ccd571a9dce),0.794


### Making recommendations

We can make recommendations using a single user or a batch of users

In [ ]:
def get_user_recommendations(user_id: int, N: int = 15, filter_liked: bool = False) -> HTML:
    
    # Get recommendations
    ids, scores = model.recommend(user_id, plays[user_id], N=N, filter_already_liked_items=filter_liked)
    
    # recommendations DataFrame
    recommendations = pd.DataFrame({
        "Artist": [format_artist(matrix_artists[i]) for i in ids],
        "Score": scores.round(3),
        "Already Listened": np.in1d(ids, plays[user_id].indices)
    })
    
    # HTML output
    content_output = f"""
    <h3>Recommendations for User {user_id}</h3>
    {recommendations.to_html(escape=False, index=False)}
    """
    
    return HTML(content_output)

In [65]:
# Recommend artists for a user
user_id = 12345
get_user_recommendations(user_id, N=15)

Artist,Score,Already Listened
Howard Shore (9b58672a-e68e-4972-956e-a8985a165a1f),1.286,True
John Williams (53b106e7-0cc6-42cc-ac95-ed8d30a3a98e),1.230,True
Nicholas Britell (afa80039-cd8a-47ae-947e-538df896586f),1.220,False
Hans Zimmer (e6de1f3b-6484-491c-88dd-6d619f142abc),1.206,False
Thomas Newman (348f3a5f-5112-4d72-b33b-11c60b4f9af2),1.190,True
Ennio Morricone (a16e47f5-aa54-47fe-87e4-bb8af91a9fdd),1.171,True
Ben Prunty (b16057eb-b7aa-4306-8f3e-430eec1f5a01),1.144,True
Ludwig Göransson (78471616-3cb4-4139-a148-aeac3c7d0f79),1.138,True
Daniel Pemberton (49c4816a-dcbe-4dfa-866a-563a51b76f48),1.137,False
Gareth Coker (96eca91a-fdb5-4a0e-908b-ba526ba166df),1.135,True


### Making batch recommendations

In [ ]:
def get_batch_recommendations(batch_size: int = 10, N: int = 5, filter_liked: bool = False) -> HTML:
    
    # Get first batch_size users
    userids = np.arange(batch_size)
    
    # Get batch recommendations
    ids, scores = model.recommend(userids, plays[userids], N=N, filter_already_liked_items=filter_liked)
    
    all_recommendations = []
    
    for user_idx, (user_artists, user_scores) in enumerate(zip(ids, scores)):
        # Create user recommendations
        user_recs = pd.DataFrame({
            "User ID": userids[user_idx],
            "Artist": [format_artist(matrix_artists[i]) for i in user_artists],
            "Score": user_scores.round(3),
            "Already Listened": np.in1d(user_artists, plays[userids[user_idx]].indices)
        })
        all_recommendations.append(user_recs)
    
    # Combine all recommendations
    combined_df = pd.concat(all_recommendations)
    
    # Create HTML output
    content_output = f"""
    <h3>Batch Recommendations for {batch_size} Users</h3>
    <p><small>Top {N} recommendations per user</small></p>
    {combined_df.to_html(escape=False, index=False)}
    """
    
    return HTML(content_output)

In [67]:
get_batch_recommendations(batch_size=5)

User ID,Artist,Score,Already Listened
0,Thievery Corporation (a505bb48-ad65-4af4-ae47-29149715bff9),1.286,True
0,Kid Francescoli (4e7473b1-0e6e-4530-b374-b9c80ec7832b),1.286,True
0,Dead Can Dance (ccda046a-2674-4f7d-97e6-f23d6c156432),1.269,True
0,Jean‐Michel Jarre (86e2e2ad-6d1b-44fd-9463-b6683718a1cc),1.266,True
0,Thylacine (e095407c-4103-4c66-b616-0dab1db05106),1.240,False
1,Taylor Swift (20244d07-534f-4eff-b4d4-930878889970),0.800,False
1,Sabrina Carpenter (1882fe91-cdd9-49c9-9956-8e06a3810bd4),0.759,True
1,Olivia Rodrigo (6925db17-f35e-42f3-a4eb-84ee6bf5d4b0),0.748,False
1,Chappell Roan (56a55378-f155-48de-80a5-d80104221267),0.739,True
1,Sort Stue (3c25958a-2bff-4381-8eb4-7dbe84c3e75e),0.719,True


## Analysis

**Most common artists from our musicbrainz data**

In [88]:
# analyzing the most common artsits in our dataset

def analyze_common_artists(n=10):
    """Analyze most common artists in the dataset"""
    df = pd.read_csv(Path(working_root) / "user_artist_playcounts.csv")
    
    # Group by artist and sum play counts
    top_artists = df.groupby(['artist_id', 'artist_name'])['play_count'].sum()\
                   .sort_values(ascending=False)\
                   .head(n)
    
    # Create formatted output
    results = pd.DataFrame(top_artists).reset_index()
    results.columns = ['Artist MBID', 'Artist Name', 'Total Plays']
    
    return results


In [89]:
# Display top artists
top_artists = analyze_common_artists()
display(top_artists)

,Artist MBID,Artist Name,Total Plays
0,20244d07-534f-4eff-b4d4-930878889970,Taylor Swift,468544
1,260b6184-8828-48eb-945c-bc4cb6fc34ca,Charli xcx,287828
2,f59c5520-5f46-4d2c-b2c4-822eabf53419,Linkin Park,280998
3,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,231478
4,381086ea-f511-4aba-bdf9-71c753dc5077,Kendrick Lamar,174880
5,f4fdbb4c-e4b7-47a0-b83b-d91bbfcfa387,Ariana Grande,170672
6,f4abc0b5-3f7a-4eff-8f78-ac078dbce533,Billie Eilish,160655
7,b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d,The Beatles,142774
8,b51c672b-85e0-48fe-8648-470a2422229f,aespa,140071
9,164f0d73-1234-4e2c-8743-d77bf2191051,Ye,131596


**Check Unique users in our data**

In [6]:
# Check unique users in final dataset
df = pd.read_csv(Path(working_root) / "user_artist_playcounts.csv")
unique_users = df['user_id'].nunique()
total_entries = len(df)

print(f"Dataset Statistics:")
print(f"Total entries: {total_entries:,}")
print(f"Unique users: {unique_users:,}")
print(f"Average entries per user: {total_entries/unique_users:.1f}")


Dataset Statistics:
Total entries: 7,947,518
Unique users: 15,780
Average entries per user: 503.6
